#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: linefragment


Variabili utili per testare il funzionamento


In [6]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using DataStructures
using OrderedCollections
using BenchmarkTools

In [7]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box
Sigma = spaceindex((V,EV))

5-element Vector{Vector{Int64}}:
 [4, 5, 2]
 [1, 3, 5]
 [4, 5, 2]
 [1, 3, 5]
 [4, 1, 3, 2]

## Versione iniziale di linefragment

Calcola le sequenze dei parametri ordinati frammentando l’input. Inoltre, i parametri di bordo (0 e 1) sono inclusi nel valore di ritorno dell’output. Il parametro ‘Sigma’ identifica un indice che fornisce un sottoinsieme di linee il cui contenuto interseca il ‘box’ di ciascuna linea di input (identificata dal parametro “EV”)

In [8]:
function linefragments(V,EV,Sigma)
	# remove the double intersections by ordering Sigma
	m = length(Sigma)
	sigma = map(sort,Sigma)
	reducedsigma = sigma ##[filter(x->(x > k), sigma[k]) for k=1:m]
	# pairwise parametric intersection
	params = Array{Float64,1}[[] for i=1:m]
	for h=1:m
		if sigma[h] ≠ []
			line1 = V[:,EV[h]]
			for k in sigma[h]
				line2 = V[:,EV[k]]
				out = Lar.intersection(line1,line2) # TODO: w interval arithmetic
				if out ≠ nothing
					α,β = out
					if 0<=α<=1 && 0<=β<=1
						push!(params[h], α)
						push!(params[k], β)
					end
				end
			end
		end
	end
	# finalize parameters of fragmented lines
	fragparams = []
	for line in params
		push!(line, 0.0, 1.0)
		line = sort(collect(Set(line)))
		push!(fragparams, line)
	end
	return fragparams
end

linefragments (generic function with 1 method)

In [22]:
@btime linefragments(V,EV,Sigma)  # 76.250 μs


  76.250 μs (822 allocations: 26.81 KiB)


5-element Vector{Vector{Float64}}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

la funzione non type unstable in quanto ho nell'output la stringa:

Body::Array{Any,1}

### versione parallelizzata linefragments


Abbiamo parallelizzato attraverso la macro @threads notando un lieve miglioramento. 

In [11]:
using Base.Threads
function linefragmentsMOD(V,EV,sigma)
    m = length(sigma) 
    sigma = map(sort,sigma) 
    params = Array{Array{Float64,1}}(undef,m)
    @threads for i=1:m
        params[i] = []
    end
    line1=[0.0 0.0; 0.0 0.0]
    line2=[0.0 0.0; 0.0 0.0]
    @threads for h=1:m
        if sigma[h] ≠ []
            line1 = V[:,EV[h]]
            @threads for k in sigma[h]
            line2 = V[:,EV[k]]
                out = intersection(line1,line2) 
                if out ≠ ()
                    if 0<=out[1]<=1 && 0<=out[2]<=1
                        push!(params[h], out[1])
                        push!(params[k], out[2])
                    end
                end
            end
        end
        end
    len = length(params)
    @threads for i=1:len
        push!(params[i], 0.0, 1.0)
        params[i] = sort(collect(Set(params[i])))
    end
    return params
end
print("Numero di threads allocati :")
println(nthreads())

Numero di threads allocati :1


In [20]:
@btime linefragmentsMOD(V,EV,Sigma)     #  34.451 μs

  34.451 μs (800 allocations: 25.34 KiB)


5-element Vector{Any}:
 [0.0, 1.0]
 [0.0, 0.5, 1.0]
 [0.0, 1.0]
 [0.0, 1.0]
 [0.0, 0.5, 1.0]

## TEST


In [13]:
using Test

@testset "linefragments Tests" begin
    V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);
    EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];
    @test Lar.spaceindex((V,EV)) == 
    [[4, 5, 2], [1, 3, 5], [4, 5, 2], [1, 3, 5], [4, 1, 3, 2]]
    Sigma = [[4, 5, 2], [1, 3, 5], [4, 5, 2], [1, 3, 5], [4, 1, 3, 2]]
    @test Lar.linefragments(V,EV,Sigma) ==
    [[0.0, 1.0], [0.0, 0.5, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 0.5, 1.0]]
end

Test Summary:       | Pass  Total
linefragments Tests |    2      2


Test.DefaultTestSet("linefragments Tests", Any[], 2, false, false)

![test di linefragment](https://github.com/MarcoCap13/LARSplitting2D/blob/main/docs/test/linefragments_test.png?raw=true)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [14]:
@benchmark linefragments(V,EV,Sigma) 

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  34.197 μs …  16.795 ms  ┊ GC (min … max): 0.00% … 99.16%
 Time  (median):     37.307 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   70.834 μs ± 346.773 μs  ┊ GC (mean ± σ):  8.34% ±  2.22%

  █▄▁   ▃▄▅▆▃▁                                                 ▁
  ████▇█████████▇█▇▆▇▆▆▆▆▅▅▅▆▅▅▅▅▄▄▅▄▄▅▅▆▃▅▃▄▃▁▄▁▄▃▁▃▃▄▁▄▁▃▄▄▅ █
  34.2 μs       Histogram: log(frequency) by time       308 μs <

 Memory estimate: 25.34 KiB, allocs estimate: 800.

funzione modificata:

In [15]:
@benchmark linefragmentsMOD(V,EV,Sigma) 

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  77.741 μs …   8.990 ms  ┊ GC (min … max): 0.00% … 98.33%
 Time  (median):     80.260 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   97.045 μs ± 166.490 μs  ┊ GC (mean ± σ):  3.62% ±  2.19%

  █▅▃▁                                                         ▁
  ██████▇█▇▇▇▇█▇▇▇▇▇█▇██▇█▇▇▆▇▆▇▆▆▅▆▆▅▅▅▅▅▅▅▄▄▅▄▅▃▃▄▄▅▄▄▃▅▃▂▂▄ █
  77.7 μs       Histogram: log(frequency) by time       300 μs <

 Memory estimate: 26.81 KiB, allocs estimate: 822.

### Funzioni di supporto aggiuntive

In [16]:
function intersection(line1,line2)
    x1,y1,x2,y2 = vcat(line1...)
    x3,y3,x4,y4 = vcat(line2...)

    det = (x4-x3)*(y1-y2)-(x1-x2)*(y4-y3)
    if det != 0.0
        a = 1/det
        b = [y1-y2 x2-x1; y3-y4 x4-x3]  # x1-x2 => x2-x1 bug in the source link !!
        c = [x1-x3; y1-y3]
        (β,α) = a * b * c
    else
        return ()
    end
    return α,β
end

function boundingbox(vertices::Lar.Points)
    minimum = mapslices(x->min(x...), vertices, dims=2)
    maximum = mapslices(x->max(x...), vertices, dims=2)
    return minimum, maximum
 end
 
 function coordintervals(coord,bboxes)
     boxdict = OrderedDict{Array{Float64,1},Array{Int64,1}}()
     for (h,box) in enumerate(bboxes)
         key = box[coord,:]
         if haskey(boxdict,key) == false
             boxdict[key] = [h]
         else
             push!(boxdict[key], h)
         end
     end
     return boxdict
 end
 
 function boxcovering(bboxes, index, tree)
     covers = [[] for k=1:length(bboxes)]
     for (i,boundingbox) in enumerate(bboxes)
         extent = bboxes[i][index,:]
         iterator = IntervalTrees.intersect(tree, tuple(extent...))
         for x in iterator
             append!(covers[i],x.value)
         end
     end
     return covers
 end
 
 function spaceindex(model::Lar.LAR)::Array{Array{Int,1},1}
            V,CV = model[1:2]
            # se il modello è in 3d o 2d (guardo le righe di V, in 3d V è una 3xN, in 2d V è una 2xN)
            dim = size(V,1)
            cellpoints = [ V[:,CV[k]]::Lar.Points for k=1:length(CV) ]
            #----------------------------------------------------------
            bboxes = [hcat(boundingbox(cell)...) for cell in cellpoints]
            xboxdict = coordintervals(1,bboxes)
            yboxdict = coordintervals(2,bboxes)
            # xs,ys are IntervalTree type
            xs = IntervalTrees.IntervalMap{Float64, Array}()
            for (key,boxset) in xboxdict
                xs[tuple(key...)] = boxset
            end
            ys = IntervalTrees.IntervalMap{Float64, Array}()
            for (key,boxset) in yboxdict
                ys[tuple(key...)] = boxset
            end
            xcovers = boxcovering(bboxes, 1, xs)
            ycovers = boxcovering(bboxes, 2, ys)
            covers = [intersect(pair...) for pair in zip(xcovers,ycovers)]
 
            if dim == 3
                zboxdict = coordintervals(3,bboxes)
                zs = IntervalTrees.IntervalMap{Float64, Array}()
                for (key,boxset) in zboxdict
                    zs[tuple(key...)] = boxset
                end
                zcovers = boxcovering(bboxes, 3, zs)
                covers = [intersect(pair...) for pair in zip(zcovers,covers)]
            end
            # remove each cell from its cover
            for k=1:length(covers)
                covers[k] = setdiff(covers[k],[k])
            end
            return covers
        end



spaceindex (generic function with 1 method)